 ECOLE NATIONALE DE LA STATISTIQUE ET DE L'ANALYSE ECONOMIQUE PIERRE NDIAYE     
        COURS DE STATISTIQUES EXPLORATOIRE ET SPATIALE - ISE1_CYCLE LONG          
                        ENSEIGNANT: MR HEMA                                     
                  TP5_GOOGLE EARTH ENGINE AVEC PYTHON                            
                 MEMBRES: FOGWOUNG DJOUFACK SARAH-LAURE                         
                          NGUEMFOUO NGOUMTSA CELINA                            
                          NIASS AHMADOU                                       
                          SENE MALICK          


In [ ]:
Consigne du TP :

    - Importer le fichier csv Points_data envoyé sur GitHub
    - Compter le nombre de points par admin
    - Créer un raster qui regroupe les différents points
    - Visualiser le raster sous forme de catégories 


In [1]:
############   STEP1: Importation des packages   ############

import ee  # c'est la bibliothèque Python qui permet d'accéder à l'API de Google Earth Engine
import geemap # Elle permet de créer et visualiser des cartes interactives basées sur des données GEE.
import pandas as pd # Elle permet la manipulation et l’analyse généralement pour des données tabulaires
from folium.plugins import Draw

# Authentification et initialisation Google Earth Engine
ee.Authenticate()  # Ceci permet de gérer le processus d'authentification pour accéder à Google Earth Engine.
ee.Initialize()   # Initialise l'instance Earth Engine pour permettre certaines opérations qu'on fera par la suite

print("DONE") # Juste pour savoir quand ca aura finit d'éxecuter 

DONE


In [6]:
############   STEP2: Importation fichier csv et visualisation ############
Donnees = ee.FeatureCollection("projects/ee-fogwoungsarahlauretp2/assets/Points_data")
    # Affichage de la première ligne pour voir comment c'est organisé
print(Donnees.first().getInfo())
    # Créeons une carte interactive centrée sur l'Afrique de l'Ouest pour mieux visualiser les données
Map = geemap.Map(center=[14.693, -17.445], zoom=4) 
    # Ajoutons la FeatureCollection à la carte
Map.addLayer(Donnees.style(**{'color': 'blue', 'pointSize': 5}), {}, 'Donnees Evenements')
Map # Affichage

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-7.499998394146455, 6.41669992512126]}, 'id': '000000000000000137d3', 'properties': {'actor1': 'Military Forces of France (1995-2007)', 'actor2': 'Private Security Forces (Ivory Coast)', 'admin1': 'Montagnes', 'admin2': 'Cavally', 'admin3': 'Guiglo', 'assoc_actor_1': '', 'assoc_actor_2': '', 'civilian_targeting': '', 'country': 'Ivory Coast', 'disorder_type': 'Political violence', 'event_date': '08 March 2003', 'event_id_cnty': 'CDI470', 'event_type': 'Battles', 'fatalities': 0, 'geo_precision': 1, 'inter1': 'External/Other forces', 'inter2': 'External/Other forces', 'interaction': 'External/Other forces-External/Other forces', 'iso': 384, 'location': 'Moyen-Cavally', 'notes': 'French troops attacked in Duekoue', 'region': 'Western Africa', 'source': 'Guardian (United Kingdom)', 'source_scale': 'International', 'sub_event_type': 'Armed clash', 'tags': '', 'time_precision': 1, 'timestamp': 1665530124, 'year': 2003}}


Map(center=[14.693, -17.445], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [35]:
############   STEP3: Comptons le nombre de points par admin ############
# Filtrons les données du Niger 
Donnees_Niger = Donnees.filter(ee.Filter.eq("country", "Niger"))
print("Total de données concernant le Niger :", Donnees_Niger.size().getInfo())

                   ############ AU NIVEAU admin1 ############
# Afficher le nombre de valeurs uniques qu'on a dans cette colonne
print("Nombre de valeurs uniques dans admin1 :", Donnees_Niger.aggregate_array('admin1').distinct().size().getInfo())
# Comptons le nombres de resultats pour chacune de ces valeurs unique
Donnees_Niger_admin1 = Donnees_Niger.aggregate_array('admin1').reduce(ee.Reducer.frequencyHistogram()).getInfo()
df_Donnees_Niger_admin1 = pd.DataFrame(list(Donnees_Niger_admin1.items()), columns=['Regions', 'Nombre_occurences']) # Convertissons sous forme de tableau 
# Affichons le tableau
print("Tableau des régions et leurs fréquences :")
print(df_Donnees_Niger_admin1)

# VERIFICATION QUE TOUT A BIEN ETE PRIS EN COMPTE
Total_admin1 = Donnees_Niger.reduceColumns(
    reducer=ee.Reducer.count(),
    selectors=["admin1"]
)
print("Total compté pour admin1:", Total_admin1.getInfo())

# Visualisation
Map2 = geemap.Map(center=[17.6078, 8.0817], zoom=6)  # Centrer sur le Niger maintenant
# Charger les données des régions 
regions = ee.FeatureCollection("projects/ee-fogwoungsarahlauretp2/assets/geoBoundaries-NER-ADM1")
# Aficher les régions sur la carte avec un style
Map2.addLayer(regions, {'color': 'blue'}, "Régions du Niger")
# Afficher les differents points par région
Map2.addLayer(Donnees_Niger, {'color': 'green'}, "Points par régions")
# Afficher la carte
Map2

Total de données concernant le Niger : 4437
Nombre de valeurs uniques dans admin1 : 8
Tableau des régions et leurs fréquences :
     Regions  Nombre_occurences
0     Agadez                392
1      Diffa               1160
2      Dosso                145
3     Maradi                398
4     Niamey                259
5     Tahoua                323
6  Tillaberi               1695
7     Zinder                 65
Total compté pour admin1: {'count': 4437}


Map(center=[17.6078, 8.0817], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [36]:
            ############ AU NIVEAU admin2 ############
# Afficher le nombre de valeurs uniques qu'on a dans cette colonne
print("Nombre de valeurs uniques dans admin2 :", Donnees_Niger.aggregate_array('admin2').distinct().size().getInfo())
# Comptons le nombres de resultats pour chacune de ces valeurs unique
Donnees_Niger_admin2 = Donnees_Niger.aggregate_array('admin2').reduce(ee.Reducer.frequencyHistogram()).getInfo()
df_Donnees_Niger_admin2 = pd.DataFrame(list(Donnees_Niger_admin2.items()), columns=['Départements', 'Nombre_occurences']) # Convertissons sous forme de tableau 
# Affichons le tableau
print("Tableau des départements et leurs fréquences :")
print(df_Donnees_Niger_admin2)

# VERIFICATION QUE TOUT A BIEN ETE PRIS EN COMPTE
Total_admin2 = Donnees_Niger.reduceColumns(
    reducer=ee.Reducer.count(),
    selectors=["admin2"]
)
print("Total compté pour admin2:", Total_admin2.getInfo())

# Visualisation
# Charger les données des régions 
departements = ee.FeatureCollection("projects/ee-fogwoungsarahlauretp2/assets/geoBoundaries-NER-ADM2")
# Aficher les departements sur la carte avec un style
Map2.addLayer(departements, {'color': 'blue'}, "Departements du Niger")
# Afficher la carte
Map2

Nombre de valeurs uniques dans admin2 : 64
Tableau des départements et leurs fréquences :
       Départements  Nombre_occurences
0             Abala                180
1            Abalak                 25
2      Aderbissinat                 20
3             Aguie                  3
4             Arlit                101
..              ...                ...
59           Tillia                120
60           Torodi                279
61  Ville de Maradi                 37
62  Ville de Niamey                259
63  Ville de Tahoua                 22

[64 rows x 2 columns]
Total compté pour admin2: {'count': 4437}


Map(bottom=7678.0, center=[17.6078, 8.0817], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

In [38]:
            ############ AU NIVEAU admin3 ############
# Afficher le nombre de valeurs uniques qu'on a dans cette colonne
print("Nombre de valeurs uniques dans admin3 :", Donnees_Niger.aggregate_array('admin3').distinct().size().getInfo())
# Comptons le nombres de resultats pour chacune de ces valeurs unique
Donnees_Niger_admin3 = Donnees_Niger.aggregate_array('admin3').reduce(ee.Reducer.frequencyHistogram()).getInfo()
df_Donnees_Niger_admin3 = pd.DataFrame(list(Donnees_Niger_admin3.items()), columns=['Communes', 'Nombre_occurences']) # Convertissons sous forme de tableau 
# Affichons le tableau
print("Tableau des communes et leurs fréquences :")
print(df_Donnees_Niger_admin3)

# VERIFICATION QUE TOUT A BIEN ETE PRIS EN COMPTE
Total_admin3 = Donnees_Niger.reduceColumns(
    reducer=ee.Reducer.count(),
    selectors=["admin3"]
)
print("Total compté pour admin3:", Total_admin3.getInfo())

# Visualisation
# Charger les données des régions 
communes = ee.FeatureCollection("projects/ee-fogwoungsarahlauretp2/assets/geoBoundaries-NER-ADM3")
# Aficher les communes sur la carte avec un style
Map2.addLayer(communes, {'color': 'blue'}, "Communes du Niger")
# Afficher la carte
Map2

Nombre de valeurs uniques dans admin3 : 164
Tableau des communes et leurs fréquences :
         Communes  Nombre_occurences
0           Abala                138
1          Abalak                 16
2    Aderbissinat                 20
3          Affala                 13
4          Agadez                132
..            ...                ...
159       Toumour                 74
160      Tounouga                  1
161     Tsernaoua                  4
162         Yelou                  4
163         Youri                  2

[164 rows x 2 columns]
Total compté pour admin3: {'count': 4437}


Map(bottom=7676.0, center=[17.644022027872726, 7.943115234375001], controls=(WidgetControl(options=['position'…

############   STEP4: Création d'un raster avec les différents points ############

Pour créer notre raster, nous allons d'abord ajouter une propriété count à chaque point, ce qui permettra de compter chaque événement. Ensuite, nous filtrerons ces points pour ne garder que ceux situés au Niger. Nous allons ensuite transformer ces points en un raster en utilisant la fonction reduceToImage(), où chaque pixel représente la somme des événements dans une zone donnée, avec une taille de pixel de 5 km. Enfin, nous afficherons ce raster sur une carte interactive pour visualiser la répartition des événements à travers le Niger.

In [47]:
pays = ee.FeatureCollection("projects/ee-fogwoungsarahlauretp2/assets/geoBoundaries-NER-ADM0")

# Ajouter une propriété numérique "count" à chaque point
Comptage_points = Donnees.map(lambda feature: feature.set('count', 1))

# Filtrer les points pour le Niger
Points_niger = Comptage_points.filter(ee.Filter.eq('country', 'Niger'))

# Créer le raster à partir des points du Niger
raster_niger = Points_niger.reduceToImage(
    properties=["count"],  # Utiliser la propriété numérique "count"
    reducer=ee.Reducer.sum()  # Réduction par somme
).reproject(
    crs="EPSG:4326",  # Système de projection WGS84
    scale=5000  # Résolution de 5 km
)

# Découper le raster avec les limites du shapefile 'pays' (qui représente le Niger)
raster_clipped_niger = raster_niger.clip(pays)





Map(center=[17.413175629395553, 9.346513612635333], controls=(WidgetControl(options=['position', 'transparent_…

In [52]:
# Les caractéristiques du raster créé
print("Informations détaillées du raster :", raster_clipped_niger.getInfo())

Informations détaillées du raster : {'type': 'Image', 'bands': [{'id': 'sum', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -9.223372036854776e+18, 'max': 9.223372036854776e+18}, 'crs': 'EPSG:4326', 'crs_transform': [0.04491576420597607, 0, 0, 0, -0.04491576420597607, 0]}]}


In [61]:
# Calculer les statistiques du raster (max, min, total)
stats = raster_clipped_niger.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=pays, 
    scale=5000, 
    maxPixels=1e8
).getInfo()

# Afficher les résultats
print("Nombre maximal de points dans un pixel : ", stats['sum_max'])
print("Nombre minimal de points dans un pixel : ", stats['sum_min'])

Nombre maximal de points dans un pixel :  254
Nombre minimal de points dans un pixel :  0


In [59]:
# Ajouter le raster découpé au Map pour visualisation
Map = geemap.Map()
Map.centerObject(pays, zoom=6)
# Palette de couleurs linéaire du bleu (petites valeurs) au rouge (grandes valeurs)
palette_couleur = ['#00008B', '#0000FF', '#00BFFF', '#00FF00', '#FFFF00', '#FFD700', '#FF6347', '#FF0000']

Map.addLayer(raster_niger_net, {
    'min': 0, 
    'max': 254, 
    'palette': palette_couleur
}, "Raster 5km - Niger (Clipped)")
Map.addLayer(pays, {'color': 'black'}, "Frontières du Niger")
Map

Map(center=[17.413175629395553, 9.346513612635333], controls=(WidgetControl(options=['position', 'transparent_…

In [63]:
############   STEP5: Visualiser le raster sous forme de catégories ############

# Définir les catégories de densité d'événements
palette_categorique = ['blue', 'green', 'yellow', 'orange', 'red']
vis_params_categorique = {
    'min': 0,
    'max': 4,
    'palette': palette_categorique
}

# Reclasser les valeurs dans des catégories
def categoriser_raster(raster):
    return raster.expression(
        "(b('sum') == 0) ? 0 : " +    # 0 événement
        "(b('sum') > 0 && b('sum') <= 5) ? 1 : " +    # 1 à 5 événements
        "(b('sum') > 5 && b('sum') <= 10) ? 2 : " +   # 6 à 10 événements
        "(b('sum') > 10 && b('sum') <= 20) ? 3 : " +  # 11 à 20 événements
        "4"                                           # Plus de 20 événements
    )

# Appliquer la classification par catégorie sur le raster
raster_categorise = categoriser_raster(raster_clipped_niger)

# Découper l'image catégorisée aux frontières du Niger
raster_categorise_niger = raster_categorise.clip(pays)

# Charger la carte
Map = geemap.Map(center=[13.5116, 2.1254], zoom=6)  # Centrer sur le Niger
Map.addLayer(raster_categorise_niger, vis_params_categorique, "Densité d'événements - Catégories")

# Afficher la carte
Map.addLayerControl()
Map


Map(center=[13.5116, 2.1254], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [81]:
#########################  REPRESENTATION SUIVANT LA CATEGORIE "DISORDER_TYPE"

# Filtrer les points en fonction des catégories de 'disorder_type'
categorie_developpements_strategiques = Donnees_Niger.filter(ee.Filter.eq('disorder_type', 'Strategic developments'))
categorie_violence_politique = Donnees_Niger.filter(ee.Filter.eq('disorder_type', 'Political violence'))
categorie_Manifestations= Donnees_Niger.filter(ee.Filter.eq('disorder_type', 'Demonstrations'))

# Compter le nombre de points dans chaque catégorie
print("Tableau des effectifs par catégorie :")
print("Developpements strategiques :", categorie_developpements_strategiques.size().getInfo())
print("Violence politique :", categorie_violence_politique.size().getInfo())
print("Demonstrations :", categorie_Manifestations.size().getInfo())

# Créer la carte et centrer sur le Niger
Map = geemap.Map()
Map.centerObject(pays, zoom=6)

# Ajouter les points pour chaque catégorie avec une couleur spécifique
Map.addLayer(categorie_developpements_strategiques, {'color': 'blue'}, "Developpements_strategiques")
Map.addLayer(categorie_violence_politique, {'color': 'red'}, "violence_politique")
Map.addLayer(categorie_Manifestations, {'color': 'yellow'}, "Manifestations")

# Ajouter les frontières du Niger pour contexte
Map.addLayer(pays, {'color': 'black'}, "Frontières du Niger")

# Afficher la carte
Map


Tableau des effectifs par catégorie :
Developpements strategiques : 1212
Violence politique : 2885
Demonstrations : 338


Map(center=[17.413175629395553, 9.346513612635333], controls=(WidgetControl(options=['position', 'transparent_…